# Practical 6 - Reviews Sentiment Classification 
In this notebook, we will attempt to classify, whether a review is good or bad, given the text of the review.  
This is the solution notebook for the [Kaggle Competition Here](https://www.kaggle.com/c/mixed-reviews-dataset/overview)

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re
import random
import string
from datetime import datetime
from functools import partial

from keras import (models, optimizers, layers, callbacks, 
                   regularizers)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import KeyedVectors

import baseline_model

%matplotlib inline

Using TensorFlow backend.


## Sourcing the data
We will use pandas to source the data from the CSV:

In [2]:
reviews_df = pd.read_csv("reviews_train.csv")

Read the test data from the CSV:

In [3]:
test_df = pd.read_csv("reviews_test.csv")

## Exploring the data
Taking a look at the first five rows of the data:

In [4]:
reviews_df.head()

,Unnamed: 0,review,label
0,0,"102 Dalmatians (2000, Dir. Kevin Lima) <br /><...",0
1,1,By 1971 it was becoming more and more obvious ...,0
2,2,"This film, once sensational for its forward-th...",0
3,3,One has to be careful whom one tells about wat...,1
4,4,I read somewhere where this film was supposed ...,0


The `Unnamed: 0` column is confusingly named, upon closer inspection, we realise that it is actually the `id`.

As such we rename `Unnamed: 0` to `id`:

In [5]:
reviews_df = reviews_df.rename(columns={"Unnamed: 0": "id"})
test_df = test_df.rename(columns={"Unnamed: 0": "id"})
reviews_df.columns

Index(['id', 'review', 'label'], dtype='object')

Lets take a look at a random review to see how the dataset works.

In [6]:
rand_idx = random.randint(0, len(reviews_df))
# Get random review and label from reviews dataframe 
rand_review = reviews_df.loc[rand_idx, "review"]
rand_label = reviews_df.loc[rand_idx, "label"]

print("Review: \n", rand_review, "\n Label: ", rand_label)

Review: 
 I bought this movie last weekend at my local Movie Gallery. It was buy 2 get 2 free and I needed one more so I chose this one. Horrible mistake. The box reads like it would be a really good movie. Well, it starts out like it is going to be this great movie. For about 5 minutes, that is. The movie is about a young woman, Laila, who gets killed trying to save her beau, Jack, from a bull. Laila's dad, Cordobes, is a rancher that the townspeople are afraid of. He assumes that Jack killed Laila because she was supposedly afraid of this bull, and goes on this hunt to find him. That was the first 5 minutes that is good. What follows after that is only gonna get 100 times worse. Whoever wrote the script, in my opinion, had to of been on some kind acid trip or something because nothing else made any sense what so ever. Jack is on the run and finds this traveling radio DJ named Mary who gives him a ride. I think Mary is supposed to be a virgin Mary type character. You know, Jesus' moth

As we can observe from the reviews, we have to do the following steps before we can conduct ML:
- Split up the words in the review
- Remove the HTML tags (ie `<br/>`)
- Deal with the punctuation somehow>
- Dealing capital letters in the reviews
- Dealing with reviews with no words whatever

## Preparing the data
Removing the HTML tags from the data as they not relevant:

In [7]:
# make a copy so that we dont overwrite the original dataframe
pp_reviews_df = reviews_df.copy()
pp_test_df = test_df.copy()
# remove html tags
pp_reviews_df["review"] = reviews_df["review"].transform(
    (lambda review: re.sub("<.+>", "", review)))
pp_test_df["review"] = test_df["review"].transform(
    (lambda review: re.sub("<.+>", "", review)))

Removing punctuation in the reviews:

In [8]:
def remove_punctuation(input_str):
    for p in string.punctuation:
        input_str = input_str.replace(p, "")
    return input_str

pp_reviews_df["review"] = pp_reviews_df["review"].transform(
     remove_punctuation)
pp_test_df["review"] = pp_test_df["review"].transform(
     remove_punctuation)

Remove the captial letters by converting them to lower case:

In [9]:
pp_reviews_df["review"] = pp_reviews_df["review"].transform(
    (lambda review: review.lower()))
pp_test_df["review"] = pp_test_df["review"].transform(
    (lambda review: review.lower()))

We have finished with the data cleaning process.  
Lets take a look at the processed data:

In [10]:
pp_reviews_df.head()

,id,review,label
0,0,102 dalmatians 2000 dir kevin lima shes change...,0
1,1,by 1971 it was becoming more and more obvious ...,0
2,2,this film once sensational for its forwardthin...,0
3,3,one has to be careful whom one tells about wat...,1
4,4,i read somewhere where this film was supposed ...,0


### Converting the Words into Vectors
#### One hot encoding
We will convert the each review into a vector of counts of each word.

In [11]:
vectorizer = CountVectorizer()
reviews = pp_reviews_df["review"]
review_vectors = vectorizer.fit_transform(reviews)

In [12]:
review_vectors.toarray()

MemoryError: 

In [13]:
print("required memory:")
len(pp_reviews_df) * len(vectorizer.get_feature_names())

required memory:


6945550000

However, One Hot Encoding the data using `CountVectorizer` does not seem very feasible because it requires a load of RAM we do not have.

### Feature Extraction using Word Vectors
Here we apply a form of transfer learning: using other peoples models to do feature extraction. In this case we use Facebook AI research's Fasttext model to convert the words to vectors

First we need to download [Fasttext pretrained word vectors](https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip) and  unzip it in the notebook's folder

Then we load the word to vector encoder like so:
> note: its going to take some time to load the vectors

In [14]:
%%time
encoder = KeyedVectors.load_word2vec_format("wiki-news-300d-1M.vec",
                                            limit=200000)

/home/zzy/.conda/envs/mlbootcamp/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 1min 9s, sys: 334 ms, total: 1min 9s
Wall time: 1min 9s


Once we have loaded the vectors, we can convert the words in the 
reviews into word vectors.

Since vector arithmetic works on word vectors, we compute the mean of the word vectors for each review to get a vector representing the "meaning" of review:

In [15]:
# define a function to convert a review into a vector
def convert_review_to_vector(review):
    # split the review into words
    words = review.split()
    
    # convert to the words into vectors
    unknown_vector = np.zeros((300,))
    vectors = [ unknown_vector ] # handle no word reviews
    
    for word in words:
        if word in encoder.vocab:
            vector = encoder[word]
        else: # word is not in encoder vocabulary
            vector = unknown_vector
        vectors.append(vector) # convert to float32 to reduce ram use

    # mean up the word vectors to get the "meaning" of the review
    return np.mean(vectors, axis=0)

# convert the reviews in the dataframe itself
%time pp_reviews_df["review"] = pp_reviews_df["review"].transform(convert_review_to_vector)
%time pp_test_df["review"] = pp_test_df["review"].transform(convert_review_to_vector)

CPU times: user 16.9 s, sys: 35.8 ms, total: 16.9 s
Wall time: 16.9 s
CPU times: user 1.02 s, sys: 11 µs, total: 1.02 s
Wall time: 1.02 s


Extract out the inputs (review vectors) and the outputs (labels):

In [16]:
review_vectors = np.stack(pp_reviews_df["review"].values) #inputs
test_review_vectors = np.stack(pp_test_df["review"].values) #inputs
labels = np.stack(pp_reviews_df["label"].values) #outputs

Split the data into the training and validation sets:

In [17]:
train_vectors, valid_vectors, train_labels, valid_labels = (
    train_test_split(review_vectors, labels, 
                     test_size=10000,
                     shuffle=True))

### Build the model
Now we proceed to build the model:

In [18]:
# Build model given parameters
def build_model(input_shape, n_outputs, scale_width, scale_depth,
               activation, l2_lambda):
    model = models.Sequential()

    # Input layer
    model.add(layers.InputLayer(input_shape))
    
    # add hidden layers
    for i in range(scale_depth):
        model.add(layers.Dense(scale_width,
                               kernel_regularizer=
                               regularizers.l2(l2_lambda)))
        model.add(activation())
    
    # output layer
    model.add(layers.Dense(n_outputs, activation="sigmoid"))
    
    return model

Lets test our build function like so:

In [19]:
model = build_model(input_shape=(300,),
                   n_outputs=1,
                   scale_width=64,
                   scale_depth=3,
                   activation=layers.ReLU,
                   l2_lambda=0)

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                19264     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_3 (ReLU)               (None, 64)                0         
_________________________________________________________________
dens

## Training the Model
Since this is a binary classification problem ('good' or 'bad'),  
we should use the `binary_crossentropy` loss function.


In [20]:
# We compile the model as follows:
model.compile(
    optimizers.Adam(lr=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"])

# derive a unique name for our run based on the current time
run_name = "run_" + datetime.strftime(datetime.now(), "%Y_%m_%d__%H_%M_%S")
run_path = os.path.join("logs", run_name)
os.makedirs(run_path, exist_ok=True)

# Training the model:
model.fit(train_vectors, train_labels,
         validation_data=(valid_vectors, valid_labels),
         batch_size=64,
         epochs=4,
         callbacks=[callbacks.TensorBoard(log_dir=run_path)])

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/4
40000/40000 [==============================] - 2s 41us/step - loss: 0.4746 - acc: 0.7711 - val_loss: 0.4188 - val_acc: 0.8090
Epoch 2/4
40000/40000 [==============================] - 1s 31us/step - loss: 0.4106 - acc: 0.8148 - val_loss: 0.4049 - val_acc: 0.8174
Epoch 3/4
40000/40000 [==============================] - 1s 31us/step - loss: 0.4025 - acc: 0.8173 - val_loss: 0.4111 - val_acc: 0.8102
Epoch 4/4
40000/40000 [==============================] - 1s 31us/step - loss: 0.3952 - acc: 0.8207 - val_loss: 0.3953 - val_acc: 0.8201


# Iterating the Model
We change different hyperparameters of the model to attempt to improve its performance (accuracy):
- learning rate
- no. of epochs trained
- no. of hidden layers
- no. of hidden units
- adding regularization

In [21]:
model = build_model(input_shape=(300,),
                   n_outputs=1,
                   scale_width=128,
                   scale_depth=2,
                   activation=(lambda:layers.LeakyReLU(0.3)),
                   l2_lambda=1e-4)

# We compile the model as follows:
model.compile(
    optimizers.Adam(lr=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"])

# derive a unique name for our run based on the current time
run_name = "run_" + datetime.strftime(datetime.now(), "%Y_%m_%d__%H_%M_%S")
run_path = os.path.join("logs", run_name)
os.makedirs(run_path, exist_ok=True)

# Training the model:
model.fit(train_vectors, train_labels,
         validation_data=(valid_vectors, valid_labels),
         batch_size=64,
         epochs=100,
         callbacks=[callbacks.TensorBoard(log_dir=run_path),
                    callbacks.ReduceLROnPlateau(monitor="val_loss",
                                               factor=0.5,
                                               patience=4,
                                               cooldown=4)])

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.4902 - acc: 0.7801 - val_loss: 0.4610 - val_acc: 0.7995
Epoch 2/100
40000/40000 [==============================] - 1s 29us/step - loss: 0.4399 - acc: 0.8125 - val_loss: 0.4326 - val_acc: 0.8152
Epoch 3/100
40000/40000 [==============================] - 1s 29us/step - loss: 0.4288 - acc: 0.8179 - val_loss: 0.4264 - val_acc: 0.8155
Epoch 4/100
40000/40000 [==============================] - 1s 29us/step - loss: 0.4248 - acc: 0.8163 - val_loss: 0.4259 - val_acc: 0.8151
Epoch 5/100
40000/40000 [==============================] - 1s 29us/step - loss: 0.4199 - acc: 0.8188 - val_loss: 0.4327 - val_acc: 0.8124
Epoch 6/100
40000/40000 [==============================] - 1s 29us/step - loss: 0.4190 - acc: 0.8198 - val_loss: 0.4273 - val_acc: 0.8123
Epoch 7/100
40000/40000 [==============================] - 1s 29us/step - loss: 0.4135 - acc: 0.8233 - val_loss: 0.4183 - 

## Predicting using the Model
We predict using the model by feeding it test data:

In [22]:
valid_probs = model.predict(valid_vectors)
test_probs = model.predict(test_review_vectors)

We threshold the probabilties to get a set of predictions:

In [23]:
threshold = 0.5
valid_preds = np.squeeze(valid_probs >= threshold)
test_preds = np.squeeze(test_probs >= threshold)

We tune threshold by cross validating using accuracy:

In [24]:
print(f"accuracy: {accuracy_score(valid_labels, valid_preds)}")

accuracy: 0.8271


## Preparing your Predictions
Prepare test predictions in the format `id`, `labels`:

In [25]:
# Obtain the ids
ids = test_df["id"].values

# Create the submission dataframe
submit_df = pd.DataFrame(data={
    "id": ids,
    "label": test_preds.astype("int")
})

submit_df.head()

,id,label
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


Finally, we write out submission dataframe into a CSV file:

In [26]:
print("Predictions: ", len(submit_df))
submit_df.to_csv("submit.csv", index=False)

Predictions:  3000
